In [1]:
import sys
sys.executable

'/Users/a200031115/opt/anaconda3/envs/mlops/bin/python'

In [3]:
pip show python

Note: you may need to restart the kernel to use updated packages.


In [4]:
print(sys.version)

3.9.15 (main, Nov 24 2022, 08:29:02) 
[Clang 14.0.6 ]


In [9]:
import pandas as pd
import numpy as np

In [7]:
wine = pd.read_csv('winequality-red.csv')

In [26]:
wine.insert(0, 'CONT_ID', range(0, 0 + len(wine)))
wine.head()

,CONT_ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [27]:
wine.head()

,CONT_ID,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [28]:
wine['quality'] = np.where(wine['quality']>6.5, 1,0)

In [29]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CONT_ID               1599 non-null   int64  
 1   fixed acidity         1599 non-null   float64
 2   volatile acidity      1599 non-null   float64
 3   citric acid           1599 non-null   float64
 4   residual sugar        1599 non-null   float64
 5   chlorides             1599 non-null   float64
 6   free sulfur dioxide   1599 non-null   float64
 7   total sulfur dioxide  1599 non-null   float64
 8   density               1599 non-null   float64
 9   pH                    1599 non-null   float64
 10  sulphates             1599 non-null   float64
 11  alcohol               1599 non-null   float64
 12  quality               1599 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 162.5 KB


In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

In [31]:
X = wine.drop('quality', axis = 1)
y = wine['quality']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [33]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, classification_report

In [35]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       320

    accuracy                           1.00       320
   macro avg       1.00      1.00      1.00       320
weighted avg       1.00      1.00      1.00       320



In [36]:
import pickle

In [37]:
with open('wine.pkl', 'wb') as f:
    pickle.dump(rfc,f)

In [38]:
import mlflow
#mlflow.set_tracking_uri("http://localhost:5000") #uncomment this line if you want to use any database like sqlite as backend storage for model
mlflow.set_experiment(experiment_name)

with mlflow.start_run():
    mlflow.sklearn.log_model(model, "model")


ModuleNotFoundError: No module named 'mlflow'

In [ ]:
experiment_name = "wine_quality_exp"
run_name="wine_quality_run"
log_model(experiment_name,run_name,run_metrics,rfc)

In [ ]:
def log_model(experiment_name,run_name,model, run_metrics=None, run_params=None):
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run():
        if not run_params == None:
            for name in run_params:
                mlflow.log_param(run_params[name])
        if not run_params == None:
            for name in run_metrics:
                mlflow.log_metric(name, run_metrics[name])
        mlflow.sklearn.log_model(model, "model")
        
    print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))